In [1]:
import pandas as pd
import re
import warnings
import os
from datetime import datetime
warnings.filterwarnings('ignore')

# ---------------------- 1. 读取原始数据 ----------------------
raw_file_path = "/Users/syx/Documents/python脚本/大众点评-爬取代码/pet_funeral_beijing.xlsx"  # 请确认文件路径正确
if not os.path.exists(raw_file_path):
    print(f"❌ 原始文件不存在：{raw_file_path}")
    exit()
df = pd.read_excel(raw_file_path)
print("="*50)
print(f"📊 原始数据概况：总行数={len(df)}，字段={df.columns.tolist()}")
print("="*50)


# ---------------------- 2. 筛选相关商家（保留原逻辑） ----------------------
def filter_pet_funeral_merchants(df):
    core_keywords = ['宠物殡葬', '宠物火化', '宠物善后', '宠物善终','壁葬','安葬','宠物天堂','善终',
                    '宠物安葬', '宠物葬礼', '宠物纪念', '毛孩子殡葬','火化','生命纪念馆']
    keyword_condition = (
        df['name'].astype(str).str.contains('|'.join(core_keywords), case=False) | 
        df['address'].astype(str).str.contains('|'.join(core_keywords), case=False) | 
        df['comment_num'].astype(str).str.contains('|'.join(core_keywords), case=False)
    )
    # 筛选后去重
    df_filtered = df[keyword_condition].drop_duplicates(subset=['name', 'address'], keep='first').reset_index(drop=True)
    return df_filtered

df_related = filter_pet_funeral_merchants(df)


# ---------------------- 3. 核心修复：准确统计“无评分（含0.0分）”的商家 ----------------------
def count_no_star_merchants(df):
    """统计所有无评分的商家（含0.0分、空值、“无评分”文本等）"""
    # 定义“无评分”的判断条件（覆盖所有场景）
    no_star_condition = (
        # 场景1：float格式的0.0分
        (df['star'] == 0.0) | 
        # 场景2：字符串格式的“0.0分”“0分”
        df['star'].astype(str).str.contains(r'^0\.0分$|^0分$', regex=True) | 
        # 场景3：空值、空字符串
        (df['star'].isna()) | (df['star'].astype(str) == '') | 
        # 场景4：明确的“无评分”文本
        (df['star'] == '无评分') | (df['star'] == '暂无评分')
    )
    # 统计符合条件的商家数量
    no_star_count = len(df[no_star_condition])
    # 返回数量和对应的商家数据（便于验证）
    no_star_merchants = df[no_star_condition]
    return no_star_count, no_star_merchants


# ---------------------- 4. 重新统计并验证 ----------------------
# 统计相关商家中的无评分数量
no_star_count, no_star_merchants = count_no_star_merchants(df_related)
# 统计无人均价格的数量（同步优化，避免类似错误）
no_avg_price_count = len(df_related[
    (df_related['avg_price'] == '人均 -') | (df_related['avg_price'].isna()) | (df_related['avg_price'].astype(str) == '')
])
# 统计有完整数据（有人均+有评分）的数量
complete_data_count = len(df_related) - no_avg_price_count - no_star_count


# ---------------------- 5. 保存筛选结果 ----------------------
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
related_file_name = f"pet_funeral_beijing_cleaned_{timestamp}.xlsx"
related_file_path = os.path.join(os.getcwd(), related_file_name)
df_related.to_excel(related_file_path, index=False, engine='openpyxl')


# ---------------------- 6. 输出准确结果 ----------------------
print("\n" + "="*60)
print(f"🎉 筛选完成！相关商家总行数：{len(df_related)}")
print(f"📁 保存路径：{related_file_path}")
print("\n📋 相关商家数据完整性统计（修复后）：")
print(f"1. 含'人均 -'/无人均的商家：{no_avg_price_count} 家")
print(f"2. 无评分（含0.0分）的商家：{no_star_count} 家")  # 修复后的统计结果
print(f"3. 同时有人均+有评分的商家：{complete_data_count} 家")
print("\n🔍 无评分商家预览（前3家，若有）：")
if no_star_count > 0:
    print(no_star_merchants[['name', 'avg_price', 'star', 'address']].head(3))
else:
    print("   暂无无评分的相关商家")
print("="*60)


# 可选：验证原始评分字段的格式（帮助你确认数据实际情况）
print(f"\n📊 原始评分字段（star）的唯一值示例：")
unique_stars = df_related['star'].unique()[:10]  # 打印前10个唯一值
for star in unique_stars:
    print(f"   格式={type(star).__name__}, 值='{star}'")

📊 原始数据概况：总行数=194，字段=['page', 'index', 'name', 'link', 'address', 'avg_price', 'comment_num', 'star']

🎉 筛选完成！相关商家总行数：60
📁 保存路径：/Users/syx/Documents/python脚本/pet_funeral_beijing_cleaned_20251113_140720.xlsx

📋 相关商家数据完整性统计（修复后）：
1. 含'人均 -'/无人均的商家：12 家
2. 无评分（含0.0分）的商家：15 家
3. 同时有人均+有评分的商家：33 家

🔍 无评分商家预览（前3家，若有）：
                   name avg_price  star  address
1       七彩桥宠物殡葬火化善终服务中心      人均 -  0.0分  草桥/公益西桥
4   宠安·宠物火化殡葬服务中心(昌平二店)      人均 -  0.0分      天通苑
7  宠安·宠物火化殡葬服务中心(博泰国际店)   人均 ￥424  0.0分       望京

📊 原始评分字段（star）的唯一值示例：
   格式=str, 值='4.0分'
   格式=str, 值='0.0分'
   格式=str, 值='5.0分'
   格式=str, 值='4.5分'
   格式=str, 值='3.0分'
   格式=str, 值='3.5分'


In [ ]:
import pandas as pd
import os
from datetime import datetime

# ---------------------- 配置参数（请修改为你的实际路径） ----------------------
INPUT_RAW_DATA = "/Users/syx/Documents/python脚本/pet_funeral_beijing_cleaned_20251110_202439.xlsx"  # 你的原始清洗数据路径
OUTPUT_CLEANED_DATA = f"beijing_second_cleaned_{datetime.now().strftime('%Y%m%d')}.xlsx"  # 输出路径


# ---------------------- 核心：北京地址精准分区函数 ----------------------
def address_to_district(address):
    """将地址映射到北京行政区（覆盖东坝、宋庄、天通苑等地点）"""
    if pd.isna(address):
        return "地址缺失"
    
    addr = str(address).strip().replace(" ", "").lower()
    if addr == "":
        return "地址缺失"

    # 北京地点-行政区映射表（严格按行政规划）
    district_mapping = {
        "朝阳区": ["朝阳区", "朝阳", "东坝", "朝外大街", "望京", "百子湾", 
                  "朝阳大悦城", "金盏", "王四营", "双井", "朝阳公园", "团结湖", "酒仙桥"],
        "海淀区": ["海淀区", "海淀", "四季青", "万柳", "颐和园", "牡丹园", "北太平庄", "远大路"],
        "昌平区": ["昌平区", "昌平", "天通苑", "回龙观", "北七家", "昌平镇"],
        "大兴区": ["大兴区", "大兴", "亦庄", "西红门", "旧宫", "黄村"],
        "通州区": ["通州区", "通州", "宋庄", "新华大街"],
        "丰台区": ["丰台区", "丰台", "夏家胡同", "纪家庙", "分钟寺", "成寿寺", 
                  "青塔", "半壁店", "云岗", "南宫"],
        "石景山区": ["石景山区", "石景山", "鲁谷", "古城", "八角"],
        "顺义区": ["顺义区", "顺义"],
        "西城区": ["西城区", "西城", "阜成门", "西直门", "动物园"],
        "东城区": ["东城区", "东城", "广渠门内"],
        "房山区": ["房山区", "房山", "良乡", "长阳镇"]
    }

    # 精准匹配
    for district, keywords in district_mapping.items():
        for kw in keywords:
            if kw in addr:
                return district

    return "待确认（需人工核对）"  # 未匹配到的地址（极少）


# ---------------------- 执行清洗分区 ----------------------
if not os.path.exists(INPUT_RAW_DATA):
    print(f"❌ 错误：未找到原始数据文件，请检查路径：{INPUT_RAW_DATA}")
else:
    # 读取原始数据
    df = pd.read_excel(INPUT_RAW_DATA)
    if "address" not in df.columns:
        print(f"❌ 错误：原始数据中未找到 'address' 列，请确认列名正确")
    else:
        # 处理address列，新增行政区列
        df["district_cn"] = df["address"].apply(address_to_district)
        
        # 统计分区结果（方便你检查准确性）
        print(f"\n📊 地址分区结果统计（共{len(df)}条数据）：")
        district_stats = df["district_cn"].value_counts()
        for dist, cnt in district_stats.items():
            print(f"   {dist}：{cnt}条（占比：{cnt/len(df)*100:.1f}%）")
        
        # 保存清洗后的数据（含district_cn列）
        df.to_excel(OUTPUT_CLEANED_DATA, index=False, engine="openpyxl")
        print(f"\n✅ 清洗完成！带行政区的数据已保存至：{OUTPUT_CLEANED_DATA}")
        print(f"   （请先检查该文件中的 'district_cn' 列，确认分区正确后，再运行热力图代码）")


📊 地址分区结果统计（共53条数据）：
   朝阳区：15条（占比：28.3%）
   昌平区：7条（占比：13.2%）
   大兴区：6条（占比：11.3%）
   丰台区：6条（占比：11.3%）
   海淀区：6条（占比：11.3%）
   通州区：4条（占比：7.5%）
   石景山区：2条（占比：3.8%）
   顺义区：2条（占比：3.8%）
   西城区：2条（占比：3.8%）
   房山区：2条（占比：3.8%）
   东城区：1条（占比：1.9%）

✅ 清洗完成！带行政区的数据已保存至：second_cleaned_20251113.xlsx
   （请先检查该文件中的 'district_cn' 列，确认分区正确后，再运行热力图代码）
